# 챕터 17: Future를 이용한 동시성

## 17.1 예제: 세 가지 스타일의 웹 내려받기
## 17.1.1 순차 내려받기 스크립트

In [ ]:
# 예제 17-2: flags.py: 순차 내려받기 스크립트. 몇몇 함수는 다른 스크립트에서 재사용할 것이다.
import os
import time
import sys

import requests # request 라이브러리를 임포한다.

POP20_CC = ('CN IN US ID BR PK NG BD RU JP'
           'MX PH VN ET EG DE IR TR CD FR').split() # 인구가 많은 순서대로 나열한 20개 국가의 ISO 3166 국가 코드를 담은 리스트

BASE_URL = 'http://flupy.org/data/flags' # 국기 이미지를 갖고 있는 웹사이트

DEST_DIR = 'downloads/' # 이미지를 갖고 있는 웹사이트

def save_flag(img, filename): # 단지 img(바이트 시퀀스)를 DEST_DIR 안의 filename으로 저장한다.
    path = os.path.join(DEST_DIR, filename)
    with open(path, 'wb') as fp:
        fp.write(img)
        
def get_flag(cc): # 국가 코드를 인수로 받아서 URL을 만들고 이미지를 내려받는다. 응답으로 내려받은 이진 시퀀스를 반환한다.
    url = '{}/{cc}/{cc}.gif'.format(BASE_URL, cc=cc.lower())
    resp = requests.get(url)
    return resp.content

def show(text): # 문자열을 출력하고 sys.stdout.flush()를 호출해서 진행 상황을 한 줄에 출력한다.
    print(text, end='')
    sys.stdout.flush()
    
def download_many(cc_list): # download_many()는 동시성 버전과 다른 핵심 부분이다.
    for cc in sorted(cc_list): # 국가코드를 알파벳순으로 반복해서, 출력할 때 순서대로 나오도록 만든다.
        image = get_flag(cc)
        show(cc)
        save_flag(image, cc.lower() + '.gif')
        return len(cc_list)

def main(download_many): # main() 함수는 download_many()를 실행하는 데 걸린 시간을 기록하고 출력한ㄷ.
    t0 = time.time()
    count = download_many(POP20_CC)
    elapsed = time.time() - t0
    msg = '\n{} flags downloaded in {:.2f}s'
    print(msg.format(count, elapsed))
    
if __name__ == '__main__':
    main(download_many) 

## 17.1.2 concurrent.futures로 내려받기

In [ ]:
# 예제 flags-threadpool.py:futures.ThreadPoolExecutor()로 스레드화된 내려받기 스크립트
from concurrent import futures

from flags import save_flag, get_flag, show, main

MAX_WORKERS = 20

def download_one(cc):
    image = get_flag(cc)
    show(cc)
    save_flag(image, cc.lower() + '.gif')
    return cc

def download_many(cc_list):
    workers = min(MAX_WORKERS, len(cc_list))
    with futures.ThreadPoolExecutor(workers) as executor:
        res = executor.map(download_one, sorted(cc_list)) # map()은 여러 스레드에 의해 download_one() 함수가 동시에 호출된다는 것을 제외하면 내장된 map()함수와 비슷하게 작동한다. 
        
    return len(list(res))

if __name__ == '__main__':
    main(download_many)

## 17.1.3 Future는 어디에 있나?

In [ ]:
# 예제 17-4 flags_threadpool_ac.py: download_many()함수 안의 executor.submit()과 futures.as_completed()로 대체하기
def download_many(cc_list):
    with futures.ThreadPoolExecutor(max_worker=3) as executor:
        to_do = []
        for cc in sorted(cc_list):
            future = executor.submit(download_one, cc) #executor.submit()은 콜러블이 실행되도록 스케줄링하고 이 작업을 나타내는 Future 객체를 반환한다.
            to_do.append(future)
            msg = 'Scheduled for {}:{}'
            print(msg.format(cc, future))
            
        results = []
        for future in futures.as_completed(to_do): # as_completed()는 Future가 완료될 때 해당 Future 객체를 생성한다.
            res = future.result()
            msg = '{} result: {!r}'
            print(msg.format(future, res))
            results.append(res)
            
    return len(results)

## 17.2 블로킹 I/O와 GIL

## 17.3 concurrent.futures로 프로세스 실행하기

* arcfout_futures.py: 순수하게 파이썬으로 구현된 RC4 알고리즘을 이용해서 149KB에서 38KB 정도 되는 바이트 배열 10개를 암호화하고 복호화한다.

* sha_futures.py: OpenSSL 라이브러리를 사용하는 표준 라이브러리 패키지 hashlib를 이용해서 1MB 바이트 배열 10개에 대한 SHA-256 해시를 계산한다.

## 17.4 Executor.map() 실험

In [ ]:
# 예제 17-6: demo_executor_map.py: ThreadPoolExecutor.map() 메서드 사용 예
from time import sleep, strftime
from concurrent import futures

def display(*args):
    print(strftime('[%H:%M:%S]'), end=' ')
    print(*args)
    
def loiter(n):
    msg = '{}loiter({}): done.'
    display(msg.format('\t'*n, n, n))
    sleep(n)
    msg = '{}loiter({}): done.'
    display(msg.format('\t'*n, n))
    return n*10

def main():
    display('Script starting.')
    executor = futures.ThreadPoolExecutor(max_worker = 3)
    results = executor.map(loiter, range(5))
    display('results:', results)
    display('Waiting for individual results:')
    for i, result in enumerate(results):
        display('results {}: {}'.format(i, result))
        
        
main()

## 17.5 진행 상황 출력하고 에러를 처리하며 내려받기

* flads2_common.py: 이 모듈은 명령행 인수를 처리하고, 시간을 측정하고, 결과를 출력하는 main() 함수를 포함해서 모든 flags2 스크립트가 공통으로 사용할 함수와 설정을 담고 있다. 이 코드는 테스트를 지원하기 위한 코드이며, 이 장에서 설명하는 내용과 직접적인 연관성 없음.
* flags2_sequential.py: 에러를 적절히 처리하며 진행 막대를 보여주는 순차 HTTP 클라이언트다. 여기에서 구현하는 download_one() 함수는 flags2_threadpool.py에서도 사용된다.
* flags2_threadpool.py: futures.ThreadPoolExecutor에 기반해서 에러 처리와 진행 막대 통합을 보여주는 동시성 HTTP 클라이언트
* flags2_asyncio.py: 스레드 예제와 기능상으로 동일하지만. asyncio와 aiohttp를 이용해서 구현한다. 이 스크립트는 18.4절 'asyncio 내려받기 스크립트 개선'에서 설명한다.

## 17.5.1 flags2 예제에서의 에러 처리

In [ ]:
# 예제 17-12 flags2_sequential.py: 내려받는 작업을 수행하는 기본적인 함수. 둘 다 flags2_threadpool.py에서 재사용되었다.
def get_flag(base_url, cc):
    url = '{}/{cc}/{cc}.gif'.format(base_url, cc=cc.lower())
    resp = requests.get(url)
    if resp.status_code != 200:
        resp.raise_for_status()
    return resp.content

def download_one(cc, base_url, verbose=False):
    try:
        image = get_flag(base_url, cc)
    except requests.exceptions.HTTPError as exc:
        res = exc.response
        if res.status_code == 404:
            status = HTTPStatus.not_found
            msg = 'not found'
        else:
            raise
    else:
        save_flag(image, cc.lower() + '.gif')
        status = HTTPStatus.ok
        msg = 'ok'
        
    if verbose:
        print(cc, msg)
        
return Result(status, cc)

In [ ]:
# 예제 17-13: flags2_sequential.py: download_many()의 순차 구현
def download_many(cc_list, base_url, verbose, max_req):
    counter = collections.Counter()
    cc_iter = sorted(cc_list)
    if not verbose:
        cc_iter = tqdm.tqdm(cc_iter)
    for cc in cc_iter:
        try:
            res = download_one(cc, base_url, verbose)
        except requests.exceptions.HTTPError as exc:
            error_msg = 'HTTP error {res.status_code} - {res.reason}'
            error_msg = error_msg.format(res=exc.response)
        except requests.exceptions.ConnectionError as exc:
            error_msg = 'Connection error'
        else:
            error_msg = ''
            status = res.status
            
        if error_msg:
            status = HTTPStatus.erro
            counter[status] += 1
            if verbose and error_msg:
                print('*** Error for {}: {}'.format(cc, error_msg))

## 17.5.2 futures.as_completed() 사용하기

In [1]:
# flags2_threadpool.py: 전체 소스 코드
import collections
from concurrent import futures

from flags2_common import main, HTTPStatus
from flags2_sequential import download_one

DEFAULT_CONCUR_REQ = 30
MAX_CONCUR_REQ = 1000

def download_many(cc_list, base_url, verbose, concur_req):
    counter = collections.Counter()
    with futures.ThreadPoolExecutor(max_workers=concur_req) as executor:
        to_do_map = {}
        for cc in sorted(cc_list):
            future = executor.submit(download_one, cc, base_url, verbose)
            to_do_map[future] = cc
        done_iter = futures.as_completed(to_do_map)
        if not verbose:
            done_iter = tqdm.tqdm(done_iter, total=len(cc_list))
        for future in done_iter:
            try:
                res = future.result()
            except requests.exceptions.HTTPError as exc:
                error_msg = 'HTTP {res.status_code} - {res.reason}'
                error_msg = error_msg.format(res=exc.repsonse)
            except requests.exceptions.ConnectionError as exc:
                error_msg = 'Connection error'
            else:
                error_msg = ''
                status = restatus
                
            if error_msg:
                status = HTTPStatus.error
            counter[status] += 1
            if verbose and error_msg:
                cc = to_do_map[future]
                print('*** Error for {}: {}'.format(cc, error_msg))
return counter

if __name__ == '__main__':
    main(download_many, DEFAULT_CONCUR_REQ, MAX_CONCUR_REQ)

SyntaxError: unexpected EOF while parsing (<ipython-input-1-e62c02d931fc>, line 41)